In [176]:
#### NOTE TO SELF: This script is not complete. It is a work in progress. ####
### TO DO: handle imbalanced datasets
### TO DO: handle feature extraction
### TO DO: handle training and testing
### TO DO: handle model evaluation


import pandas as pd
import numpy as np
import glob
import category_encoders as ce
from pandas import DataFrame

pd.set_option('future.no_silent_downcasting', True)
# Get the file paths for all CSV files in the current folder
file_paths = glob.glob('*.csv')

## Read all CSV files into a list of DataFrames
datasets: list[DataFrame] = []
for file_path in file_paths:
    datasets.append(pd.read_csv(file_path))
    


In [177]:
### categorical features with more than 7 unique values will be encode with binary encoding
dataset_one_hot_encoded : list[list[str]] = [[] for i in range(len(datasets))]
dataset_binary_encoded: list[list[str]]   = [[] for i in range(len(datasets))]

In [178]:
## check some information about the dataset
## check if there is any null value and replace it with np.nan
## drop the examples with na values
## check the unique counts of each feature in each dataset and decide if it should be one hot encoded or binary encoded

for (index,dataset) in enumerate(datasets):
    ### replace ' ' with np.nan and drop examples with na values
    dataset=dataset.replace({' ': np.nan})
    dataset=dataset.replace({'': np.nan})
    dataset=dataset.dropna()
    
    
    print("The unique counts of each feature in this dataset are:")
    print("\n")    
    for column in dataset.columns:
        ### print information about the column
        print(column, dataset[column].dtype, dataset[column].nunique())
        ### cast the column to float if possible ( columns with ' ' has dtype object but now the na are removed we can cast it back to float)
        
        ### check if the column is categorical and if it has more than 7 unique values pass it to binary encoding, else to one hot encoding    
        if dataset[column].dtype=='object':
             ### cast the column to float if possible ( columns with ' ' has dtype object but now the na are removed we can cast it back to float)
            try:
                dataset[column] = dataset[column].astype(float)
            except ValueError:
                pass
            if dataset[column].nunique() > 7:
                dataset_binary_encoded[index].append(column)
            else:
                dataset_one_hot_encoded[index].append(column)
    print("\n\n")
    
    datasets[index] = dataset

The unique counts of each feature in this dataset are:


state object 51
account_length int64 215
area_code object 3
international_plan object 2
voice_mail_plan object 2
number_vmail_messages int64 46
total_day_minutes float64 1843
total_day_calls int64 120
total_day_charge float64 1843
total_eve_minutes float64 1773
total_eve_calls int64 123
total_eve_charge float64 1572
total_night_minutes float64 1757
total_night_calls int64 128
total_night_charge float64 992
total_intl_minutes float64 168
total_intl_calls int64 21
total_intl_charge float64 168
number_customer_service_calls int64 10
churn object 2



The unique counts of each feature in this dataset are:


CustomerID object 1869
Count int64 1
Country object 1
State object 1
City object 833
Zip Code int64 1190
Lat Long object 1190
Latitude float64 1190
Longitude float64 1190
Gender object 2
Senior Citizen object 2
Partner object 2
Dependents object 2
Tenure Months int64 72
Phone Service object 2
Multiple Lines object 3
Internet Servi

In [179]:
#check the dataset again
for dataset in datasets:
    print(dataset.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4250 entries, 0 to 4249
Data columns (total 20 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   state                          4250 non-null   object 
 1   account_length                 4250 non-null   int64  
 2   area_code                      4250 non-null   object 
 3   international_plan             4250 non-null   object 
 4   voice_mail_plan                4250 non-null   object 
 5   number_vmail_messages          4250 non-null   int64  
 6   total_day_minutes              4250 non-null   float64
 7   total_day_calls                4250 non-null   int64  
 8   total_day_charge               4250 non-null   float64
 9   total_eve_minutes              4250 non-null   float64
 10  total_eve_calls                4250 non-null   int64  
 11  total_eve_charge               4250 non-null   float64
 12  total_night_minutes            4250 non-null   f

In [180]:
### Scale the data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
for (index,dataset) in enumerate(datasets):
    for column in dataset.columns:
        if column not in ['Churn', 'Exited']:
            if dataset[column].dtype in ['float64','int64']:
                dataset[column] = scaler.fit_transform(dataset[[column]])
    datasets[index] = dataset

In [181]:
from category_encoders import OneHotEncoder, BinaryEncoder

for (index, dataset) in enumerate(datasets):
    for one_hot_column in dataset_one_hot_encoded[index]:
        encoder = OneHotEncoder(cols=[one_hot_column])
        dataset = encoder.fit_transform(dataset)
    for binary_column in dataset_binary_encoded[index]:
        encoder = BinaryEncoder(cols=[binary_column])
        dataset = encoder.fit_transform(dataset)
    datasets[index] = dataset
        
        
        
    

In [182]:
for dataset in datasets:
    print(dataset.info())
    print("\n\n")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4250 entries, 0 to 4249
Data columns (total 30 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   state_0                        4250 non-null   int64  
 1   state_1                        4250 non-null   int64  
 2   state_2                        4250 non-null   int64  
 3   state_3                        4250 non-null   int64  
 4   state_4                        4250 non-null   int64  
 5   state_5                        4250 non-null   int64  
 6   account_length                 4250 non-null   float64
 7   area_code_1                    4250 non-null   int64  
 8   area_code_2                    4250 non-null   int64  
 9   area_code_3                    4250 non-null   int64  
 10  international_plan_1           4250 non-null   int64  
 11  international_plan_2           4250 non-null   int64  
 12  voice_mail_plan_1              4250 non-null   i

In [183]:
## split feautures and labels for each dataset
X_data: list[DataFrame] = []
y_data: list[DataFrame] = []
label_column : list[list[str]] = [
    ['churn_1','churn_2'],
    ['Churn Value'],
    ['Churn'],
    ['Churn_1','Churn_2'],
    ['Exited'],
 
 ]

for (index, dataset) in enumerate(datasets):
    X_data.append(dataset.drop(columns=label_column[index]))
    y_data.append(dataset[label_column[index]])

In [184]:
### Split train and test data

from sklearn.model_selection import train_test_split

X_train: list[DataFrame] = []
y_train: list[DataFrame] = []
X_test: list[DataFrame] = []
y_test: list[DataFrame] = []

for (X,y) in zip(X_data,y_data):
    X_train_temp, X_test_temp, y_train_temp, y_test_temp = train_test_split(X, y, test_size=0.2, random_state=42)
    
    X_train.append(X_train_temp)
    y_train.append(y_train_temp)
    X_test.append(X_test_temp)
    y_test.append(y_test_temp)

In [185]:
##### DATASET SPECIFIC PREPROCESSING HERE #####
##### DATASET SPECIFIC PREPROCESSING HERE #####
##### DATASET SPECIFIC PREPROCESSING HERE #####
##### DATASET SPECIFIC PREPROCESSING HERE #####
##### DATASET SPECIFIC PREPROCESSING HERE #####

In [186]:
### PREPROCESSING FINISHED ###

In [188]:

### SOME RANDOM BS BECAUSE BUGS, IDEK :SKULLS: ###
ce2=ce.OneHotEncoder(cols=['Exited'])
y_train[4]=ce2.fit_transform(y_train[4])
ce1=ce.OneHotEncoder(cols=['Churn'])
y_train[2]=ce1.fit_transform(y_train[2])

In [190]:
#Calculate the most impactful features

from sklearn.ensemble import RandomForestClassifier
i=0
for (X,y) in zip(X_train,y_train):
    rf = RandomForestClassifier()

    # Fit the classifier to the data
    rf.fit(X, y)

    # Get feature importances
    feature_importances = rf.feature_importances_

    # Create a dictionary of feature importance scores
    feature_importance_dict = dict(zip(X.columns, feature_importances))

    # Sort the features by importance in descending order
    sorted_features = sorted(feature_importance_dict.items(), key=lambda x: x[1], reverse=True)

    # Set the number of most impactful features to keep
    n = 5

    # Get the n most impactful features
    most_impactful_features = [feature for feature, importance in sorted_features[:n]]

    # Print the most impactful features
    print("Most impactful features:")
    for feature in most_impactful_features:
        print(feature)


Most impactful features:
total_day_minutes
total_day_charge
number_customer_service_calls
total_eve_minutes
total_eve_charge
End of 0 dataset
Most impactful features:
CustomerID_0
CustomerID_1
CustomerID_2
CustomerID_3
CustomerID_4
End of 1 dataset


c:\Users\hung\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Most impactful features:
DayMins
MonthlyCharge
CustServCalls
OverageFee
RoamMins
End of 2 dataset
Most impactful features:
tenure
MonthlyCharges
Contract_1
OnlineSecurity_1
PaymentMethod_1
End of 3 dataset
Most impactful features:
Age
NumOfProducts
Balance
CreditScore
EstimatedSalary
End of 4 dataset


In [ ]:
### USE PCA TO REDUCE DIMENSIONALITY

from sklearn.decomposition import PCA

pca = PCA(n_components=5)

X_train_pca: list[DataFrame] = []
X_test_pca: list[DataFrame] = []

for (X_train_temp, X_test_temp) in zip(X_train, X_test):
    X_train_pca.append(pca.fit_transform(X_train_temp))
    X_test_pca.append(pca.transform(X_test_temp))
    

In [ ]:
### PROPOSE SOME MODEL AND USE THEM TO THEN ON ALL THE DATASETS

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score

models = [RandomForestClassifier(), GradientBoostingClassifier(), LogisticRegression(), SVC(), KNeighborsClassifier()]


# Train the model here


In [ ]:
### Calculate F1 score, confusion matrix, AUROC, AUPRC, and accuracy for each model
from sklearn.metrics import f1_score, confusion_matrix, roc_auc_score, average_precision_score